# 儲存查詢歷史

In [ ]:
import pymysql
import os
from dotenv import load_dotenv
from datetime import datetime

# 載入資料庫設定
load_dotenv()
DB_CONFIG = {
    "host": os.getenv("DB_HOST"),
    "port": int(os.getenv("DB_PORT", 3306)),
    "user": os.getenv("DB_USER"),
    "password": os.getenv("DB_PASSWORD"),
    "database": os.getenv("DB_NAME")
}

def sync_keywords_from_history(db_config):
    conn = pymysql.connect(
        **db_config,
        charset='utf8mb4',
        cursorclass=pymysql.cursors.DictCursor
    )

    with conn:
        with conn.cursor() as cursor:
            # 若尚未存在則建立 search_keywords 表
            cursor.execute("""
                CREATE TABLE IF NOT EXISTS search_keywords (
                    id INT AUTO_INCREMENT PRIMARY KEY,
                    keyword VARCHAR(255) UNIQUE,
                    last_searched DATETIME
                );
            """)

            # 從 coupang_products 抓取不同關鍵字及最新時間
            cursor.execute("""
                SELECT search_keyword AS keyword, MAX(timestamp) AS last_searched
                FROM coupang_products
                GROUP BY search_keyword
            """)
            keyword_rows = cursor.fetchall()

            print(
                f"🔍 共發現 {len(keyword_rows)} 個歷史關鍵字，準備寫入..."
            )

            for row in keyword_rows:
                keyword = row["keyword"]
                last_searched = row["last_searched"]

                # 寫入或更新，如果已存在則更新最後時間
                cursor.execute("""
                    INSERT INTO search_keywords (keyword, last_searched)
                    VALUES (%s, %s)
                    ON DUPLICATE KEY UPDATE last_searched = VALUES(last_searched)
                """, (keyword, last_searched))

        conn.commit()
        print("✅ 歷史關鍵字同步完成")

        # 顯示所有關鍵字，依時間排序
        with conn.cursor() as cursor:
            cursor.execute("""
                SELECT keyword, last_searched
                FROM search_keywords
                ORDER BY last_searched DESC
            """)
            all_keywords = cursor.fetchall()

            print("\n📋 歷史查詢關鍵字：")
            for idx, row in enumerate(all_keywords, 1):
                print(
                    f"{idx}. {row['keyword']}"
                    f"（最後查詢時間：{row['last_searched']}）"
                )

# 執行補寫動作
if __name__ == "__main__":
    sync_keywords_from_history(DB_CONFIG)

🔍 共發現 12 個歷史關鍵字，準備寫入...
✅ 歷史關鍵字同步完成

📋 歷史查詢關鍵字：
1. 味丹 MORE氣泡水（最後查詢時間：2025-05-19 09:11:14）
2. INABA CIAO 啾嚕（最後查詢時間：2025-05-19 09:10:44）
3. BLUE BAY 倍力 Sense（最後查詢時間：2025-05-19 09:10:15）
4. BLUE BAY 倍力 Animate（最後查詢時間：2025-05-19 09:09:45）
5. kose 洗面乳（最後查詢時間：2025-05-17 19:29:19）
6. uno 洗面乳（最後查詢時間：2025-05-17 16:44:31）
7. 法國皇家（最後查詢時間：2025-05-17 00:29:43）
8. ROYAL CANIN（最後查詢時間：2025-05-17 00:29:15）
9. 法國皇家 ROYAL CANIN（最後查詢時間：2025-05-17 00:22:59）
10. 味丹 氣泡水（最後查詢時間：2025-05-16 23:06:12）
11. YOLOMi 悠樂米（最後查詢時間：2025-05-12 19:18:12）
12. DORO CAT 豆乳貓（最後查詢時間：2025-05-12 00:45:26）
